<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="400" alt="cognitiveclass.ai logo"  />
</center>

# Analyze how to work with various financial indicators

Estimated time needed: **30** minutes

## Objectives

After completing this lab you will be able to:

*   Be confident about your data analysis skills


You will need the following libraries:


In [ ]:
! mamba install pandas -y
! mamba install numpy -y
! mamba install scikit-learn -y
! mamba install ipywidgets -y
! mamba install tqdm -y
! mamba install seaborn -y
! mamba install skipy -y

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split


pd.set_option("display.precision", 2)
pd.options.display.float_format = '{:.3f}'.format

This function will download the dataset into your browser 


<b>Importing the Data</b>


you will need to download the dataset; if you are running locally, please comment out the following 


In [ ]:
path = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-GPXX0L9FEN/clean_df%20(5).csv'

Load the csv:


In [ ]:
df= pd.read_csv(path, index_col=0)
df.index = pd.to_datetime(df.index)
df

We use the method  <code>head()</code>  to display the first 5 columns of the dataframe:


## Parabolic Stop and Reverse (PSAR)

Parabolic Stop and Reverse - is a technical indicator used to analyze the trend in the market. It was first developed by engineer J. Welles Wilder in 1978.

PSAR provides signals for entering and exiting positions in the market. The indicator shows where the market may change its direction and provides information about a potential Stop Loss level.

PSAR uses an algorithm that takes into account price and time to determine the entry and exit points of a position. The indicator takes the form of dots that can be located either above or below the prices, depending on the direction of the trend. If the price crosses the dots from below upwards, this may indicate a change in the trend from low to high, and if the dots cross the price from above downwards, this may indicate a change in the trend from high to low.

PSAR also helps to determine the Stop Loss level, as the indicator is located close to the price that may become the exit point of the position. If the price crosses the PSAR dot, this can be a signal to close the position and preserve profits or minimize losses.

Considering all of the above, PSAR is a useful tool for technical analysis and risk management in the market."

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #1: </h1>
<b>Calculate the PSAR and add "PSAR" to the dataframe.And also visualize the result.</b>
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute 


In [ ]:
# Configuring PSAR parameters
af_initial = 0.02
af_max = 0.2
af = af_initial
long_position = True
psar = []

# Calculation of PSAR values
for i in range(len(df)):
    if i == 0:
        psar.append(df['price BID'][i] - (df['price ASK'][i] - df['price BID'][i]))
        ep = df['price ASK'][i]
        continue

    if long_position:
        if df['price BID'][i] < psar[i-1]:
            long_position = False
            psar.append(ep)
            af = af_initial
            ep = df['price BID'][i]
            continue
        else:
            psar.append(psar[i-1] + af * (ep - psar[i-1]))
            if df['price ASK'][i] > ep:
                ep = df['price ASK'][i]
                af = min(af + af_initial, af_max)
    else:
        if df['price ASK'][i] > psar[i-1]:
            long_position = True
            psar.append(ep)
            af = af_initial
            ep = df['price ASK'][i]
            continue
        else:
            psar.append(psar[i-1] + af * (ep - psar[i-1]))
            if df['price BID'][i] < ep:
                ep = df['price BID'][i]
                af = min(af + af_initial, af_max)

# Adding PSAR to the existing dataframe
df['PSAR'] = psar
fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(df['price BTC'], label='price')
ax.plot(df['PSAR'], label='PSAR')
ax.legend(loc='upper left')
ax.set_xlabel("Date")
ax.set_ylabel('Price')
plt.show()


## Keltner Channel

Keltner Channel is a technical indicator used to determine market trend and price volatility. It consists of three lines - the middle line, upper, and lower channels - which are calculated based on the average price over a period and additional calculation based on the standard deviation of prices from the average.

Keltner channels allow traders to receive signals for entering and exiting positions based on a violation of price behavior in relation to the upper and lower channels. When the price crosses the upper channel, it may indicate that the market is overbought and there may be opportunities to exit the position. When the price crosses the lower channel, it may indicate that the market is oversold and there may be opportunities to enter a position.

Typically, traders have the ability to adjust the number of periods used to calculate the average line and standard deviation to tailor the indicators to their own needs.

 <div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #2: </h1>
<b>Calculate the Keltner Channel and add result to the dataframe.And also visualize the result.</b>
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute 


In [ ]:
n = 20
multiplier = 2

# Розрахунок середньої ціни
avg_price = (df["price ASK"] + df["price BID"]) / 2

# Розрахунок середньої ціни за n днів
avg_price_n = avg_price.rolling(window=n, min_periods=1).mean()

# Розрахунок середнього діапазону цін за n днів
range_price_n = (df["price ASK"] - df["price BID"]).rolling(window=n, min_periods=1).mean()

# Розрахунок верхньої та нижньої межі Keltner Channel
upper_band = avg_price_n + multiplier * range_price_n
lower_band = avg_price_n - multiplier * range_price_n

# Створення DataFrame з результатами
kc_df = pd.DataFrame({"Upper Band": upper_band, "Lower Band": lower_band})
kc_df.index = df.index

# Об'єднання kc_df зі старим df
df = pd.concat([df, kc_df], axis=1)

# Видалення рядків з пропущеними значеннями
df.dropna(inplace=True)

# Виведення результатів
print(df.head())

fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(df.index, df["price ASK"], label="Price ASK")
ax.plot(df.index, df["price BID"], label="Price BID")
ax.plot(df.index, df["Upper Band"], label="Upper Band")
ax.plot(df.index, df["Lower Band"], label="Lower Band")

ax.set_xlabel("Date")
ax.set_ylabel("Price")
ax.legend(loc="best")

plt.show()


## Money Flow Index

The Money Flow Index (MFI) is a technical indicator used to measure buying and selling pressure in the stock market. It takes into account both price and volume data to determine whether a stock is being accumulated (bought) or distributed (sold) by investors.

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1>Question #3:</h1>

<b>Calculate the MFI using the algorithm of actions that I have given you in a line below.</b>

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute 


In [ ]:
def calculate_mfi(df, period):
    # Calculate typical price
    tp = (df['price ASK'] + df['price BID']) / 2
    
    # Calculate money flow
    mf = tp * (df['size ASK'] + df['size BID'])
    
    # Calculate positive and negative money flow
    pmf = pd.Series(0, index=df.index)
    pmf[df['price ASK'] > df['price ASK'].shift(1)] = mf
    nmf = pd.Series(0, index=df.index)
    nmf[df['price BID'] < df['price BID'].shift(1)] = mf
    
    # Calculate money ratio
    mr = pmf.rolling(period).sum() / nmf.rolling(period).sum()
    
    # Calculate MFI
    mfi = 100 - (100 / (1 + mr))
    
    # Return MFI values as a new DataFrame
    return pd.DataFrame({'MFI': mfi})

# Calculate MFI for a 14-day period using the df DataFrame
mfi = calculate_mfi(df, 14)

# Add MFI values to the DataFrame and remove any missing data
df['MFI'] = mfi
df.dropna(inplace=True)

# Print MFI values
print(df['MFI'])

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #4a: </h1>

<p>Find the correlation between the BTC currency price and its size columns.</p>
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute
corr = df[['price BTC', 'size BTC']].corr()
corr

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #4b: </h1>

<p>Build heatmap to understand the resuluts more precisely.</p>
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute
sns.heatmap(corr, annot=True)

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1>Question #5:</h1>

<p>Given the correlation results between "price BTC" and "size BTC", do you expect a linear relationship?</p> 
<p>Verify your results using the function <code>sns.regplot()</code>.</p>
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute
sns.regplot(x='price BTC', y='size BTC', data=df)

<h4>Before proceeding to the fifth question, activate the following line</h4>

In [ ]:
X = df[['Saldo']]
Y = df['price BTC']

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1>Question #6a: </h1>

<b>Create a linear regression object called "lm1".</b>

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute 
lm1 = LinearRegression()
lm1

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #6b : </h1>

<b>Train the model using "Saldo" as the independent variable and "price BTC" as the dependent variable?</b>

</div>

In [ ]:
lm1.fit(df[['Saldo']], df[['size ASK']])
lm1

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1>Question #6c:</h1>

<b>Find the slope and intercept of the model.</b>

</div>


<h4>Slope</h4>

In [ ]:
# Write your code below and press Shift+Enter to execute 
lm1.coef_

<h4>Intercept</h4>

In [ ]:
# Write your code below and press Shift+Enter to execute 
lm1.intercept_

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1>Question #6d: </h1>

<b>What is the equation of the predicted line? You can use  "Saldo" and "price BTC".</b>

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute 
Price=0.38 + 465.42*df['Saldo']
Price

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1>Question #7: </h1>

<b>Calculate the dependence of Saldo on MFI, PSAR and Keltner Channel</b>

</div>


In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm

X = df[['PSAR', 'MFI', 'Lower Band', 'Upper Band']]
y = df['Saldo']


Rsqu_test = []
Rsqu_train = []
dummy1 = []
Alpha =  np.array(range(0,1000,10))
pbar = tqdm(Alpha)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=False)
x_train_pr = x_train[['PSAR', 'MFI', 'Lower Band', 'Upper Band']]
x_test_pr = x_test[['PSAR', 'MFI', 'Lower Band', 'Upper Band']]

for alpha in pbar:
    RigeModel = Ridge(alpha=alpha)
    RigeModel.fit(x_train_pr, y_train)
    test_score, train_score = RigeModel.score(x_test_pr, y_test), RigeModel.score(x_train_pr, y_train)
    pbar.set_postfix({"Test Score": test_score, "Train Score": train_score})
    Rsqu_test.append(test_score)
    Rsqu_train.append(train_score)
 

In [ ]:
width = 12
height = 10
plt.figure(figsize=(width, height))

plt.plot(Alpha, Rsqu_test, label='test score')
plt.plot(Alpha, Rsqu_train, label='train score')
plt.xlabel('alpha')
plt.ylabel('R-squared')
plt.legend()
plt.show()

In [ ]:
ridge_model = Ridge(alpha=10)
ridge_model.fit(x_train_pr, y_train)

y_pred = ridge_model.predict(x_test_pr)
fig, ax = plt.subplots(figsize=(width, height))
ax.plot(y_test, label = "Real Data")
ax.plot(y_test.index, y_pred, label = "Ridge Data")
ax.set_xlabel('Date')
ax.legend(loc='upper left')
import matplotlib.dates as mdates
myFmt = mdates.DateFormatter('%m.%d %H:%M')
ax.xaxis.set_major_formatter(myFmt)
plt.show()

<a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/share-notebooks.html/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDA0101ENSkillsNetwork20235326-2021-01-01"> CLICK HERE</a> to see how to share your notebook


### Thank you for completing this lab!

## Author

<a href="https://author.skills.network/instructors/danyil_zhupnyk">Danyil Zhupnyk</a><br>
<a href="https://author.skills.network/instructors/yaroslav_vyklyuk_2">Prof. Yaroslav Vyklyuk, DrSc, PhD</a><br>
<a href="https://author.skills.network/instructors/mariya_fleychuk">Prof. Mariya Fleychuk, DrSc, PhD</a><br>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                 |
| ----------------- | ------- | ---------- | ---------------------------------- |
| 2023-04-01        | 1.0     | D.Zhupnyk  | Created Lab                        |

<hr>

## <h3 align="center"> © IBM Corporation 2023. All rights reserved. <h3/>
